# OSU Micro Benchmarks (OMB)

Results of selected latency and bandwidth tests from the [OMB suite](http://mvapich.cse.ohio-state.edu/benchmarks/).

All these tests use two nodes:

- osu_bw: Single process per node
- osu_latency: Single process per node
- osu_bibw (Bidirectional Bandwidth Test): Single process per node
- osu_mbw_mr (Multiple Bandwidth / Message Rate Test): Multiple processes per node, communicating in pairs split across nodes


In [ ]:
# Plotting setup
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import ticker
from IPython.display import display, HTML

import pandas as pd
import os, sys
from pprint import pprint
sys.path.extend(('../../reframe', '../../')) # the `modules` package imports `reframe` so need that in sys.modules
import modules

# Bandwidth/latency vs message size

The plots below show results vs message size for tests using a single process per node. Results are for the most-recent test on each system/partition/environment combination.

In [ ]:
for test in ['Osu_bw', 'Osu_bibw', 'Osu_mbw_mr_2', 'Osu_latency']:
    
    # find all data and work out differences between them:
    paths = modules.utils.find_run_outputs(root='../../output', test=test)
    fig, ax = plt.subplots(nrows=1, ncols=1)
    meta = [modules.utils.parse_path_metadata(p) for p in paths]
    
    # load and plot:
    for ix, path in enumerate(paths):
        data = modules.omb.read_omb_out(path)
        label = '%s:%s' % (meta[ix]['sysname'], meta[ix]['partition'])
        plt.plot(data.columns[0], data.columns[1], 'o-', data=data, label=label)
        
    # format plot:
    ax.set_xscale('log', basex=2)
    if 'latency' in test:
        ax.set_yscale('log', basey=10)
    ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: modules.utils.sizeof_fmt(x)))
    ax.grid(True, which="both")
    ax.set_xlabel(data.columns[0])
    ax.set_ylabel(data.columns[1])
    ax.set_title('%s: %s' % (test, data.columns[1]))
    ax.legend()

# Max Bandwidth/Message Rate vs Number of Processes per Node

The plots below show the **maximum** bandwidth and message rate for tests using varying numbers of processes per node. Results are for the most-recent test on each system/partition/environment combination.

In [ ]:
# TODO: needs to be loaded from system config somewhere
Gb_to_MB = 125.0

test = 'Osu_mbw_mr_*'
    
for perf_var in ['max_bandwidth', 'max_message_rate']:
    
    # get, plot and tabulate data:
    df = modules.utils.tabulate_last_perf(test, 'procs_per_node', perf_var)
    print('%s: %s' % (perf_var, test))
    display(HTML(df.to_html()))
    ax = df.plot(grid=True, style='o-')
    
    # add max bandwidth to relevant plots:
    if 'bandwidth' in perf_var:
        xlim = ax.get_xlim()
        for syspart, bw in modules.utils.get_sys_param('interconnect_Gb').items():
            theoretical_bw_MB = bw * Gb_to_MB
            label = syspart + " (theoretical b/w)"
            ax.plot(xlim, [theoretical_bw_MB, theoretical_bw_MB], '--', label=label)
    # format:
    ax.set_xlabel('processes per node')
    ax.set_ylabel(perf_var)
    plt.tight_layout()
    ax.set_title('%s: %s' % (test, perf_var))
    ax.legend()
